- **Use 3-Level Relevance Generation (RG-3L) from "Beyond Yes and No: Improving Zero-Shot LLM Rankers via Scoring Fine-Grained Relevance Labels"**
- **Allow for multiple Evidence citations**


In [ ]:
from langchain_ollama import OllamaLLM
from tqdm import tqdm

import json
import os
import pandas as pd
import re

llm = OllamaLLM(model="llama3.2")

# Check if processed file exists
if os.path.exists('prospectuses_data_processed.csv'):
    df = pd.read_csv('prospectuses_data_processed.csv')
else:
    df = pd.read_csv('prospectuses_data.csv')
    # Filter out rows that have "failed parsing" in the Section ID column
    df = df[df['Section ID'] != "failed parsing"]
    
df = pd.read_csv('prospectuses_data.csv')

# Filter out rows that have "failed parsing" in the Section ID column
df = df[df['Section ID'] != "failed parsing"]


questions = {
    "Market Dynamics - a": "Is the company exposed to risks associated with cyclical products?",
    "Market Dynamics - b": "Does the text mention risks related to demographic or structural trends affecting the market?",
    "Market Dynamics - c": "Does the text discuss risks due to seasonal volatility in the industry?"
}

# Define the questions corresponding to each column
questions_original = {
    "Market Dynamics - a": "Exposure to cyclical products",
    "Market Dynamics - b": "Impact of demographic and structural trends",
    "Market Dynamics - c": "Seasonal industry volatility"
}

# Ensure the relevance and evidence columns are created with a compatible data type
for column_name in questions.keys():
    if column_name in df.columns:
        df[column_name] = df[column_name].astype('string')
    else:
        df[column_name] = ""

df.head(4)

,Prospectus ID,Original Filename,Section ID,Section Title,Subsection ID,Subsection Title,Subsubsection ID,Subsubsection Title,Subsubsection Text,Market Dynamics - a,Market Dynamics - b,Market Dynamics - c,LLM Answer,Evidence Text,Parsing Error
3,4,FR0014001YE4.pdf,1,RISK FACTORS,1.1,1. Risks related to the Issuer,1.1.1,NaN,Risk factors relating to the Issuer and the Gr...,<NA>,<NA>,<NA>,NaN,NaN,NaN
4,4,FR0014001YE4.pdf,1,RISK FACTORS,1.2,2. Risks related to the Bonds 2.1 Risks relati...,1.2.1,2.1.1 The Bonds may be redeemed prior to maturity,The Issuer reserves the right to purchase Bond...,<NA>,<NA>,<NA>,NaN,NaN,NaN
5,4,FR0014001YE4.pdf,1,RISK FACTORS,1.2,2. Risks related to the Bonds 2.1 Risks relati...,1.2.2,2.1.2 Change of control put option,"In accordance with each Condition 4(d), upon t...",<NA>,<NA>,<NA>,NaN,NaN,NaN
6,4,FR0014001YE4.pdf,1,RISK FACTORS,1.2,2. Risks related to the Bonds 2.1 Risks relati...,1.2.3,2.1.3 Interest rate risks,As provided for in Condition 3 of the Terms an...,<NA>,<NA>,<NA>,NaN,NaN,NaN


In [40]:
def extract_fields(response):
    # Remove any newlines and extra spaces
    response = ' '.join(response.strip().split())

    # Extract the Relevance field
    relevance_match = re.search(r'"Relevance"\s*:\s*"([^"]+)"', response)
    if relevance_match:
        relevance = relevance_match.group(1).strip()
    else:
        relevance = "Parsing Error"

    # Extract the Evidence field(s)
    evidence_match = re.search(r'"Evidence"\s*:\s*(.+?)(?:,?\s*"[^"]+"\s*:|\s*}$)', response)
    if evidence_match:
        evidence_str = evidence_match.group(1).strip()
        # Remove any trailing commas or braces
        evidence_str = evidence_str.rstrip(', }')
        # Split the evidence_str into individual evidence items
        # Evidence items are strings enclosed in double quotes
        evidence_items = re.findall(r'"([^"]+)"', evidence_str)
        evidence = evidence_items
    else:
        evidence = []

    return relevance, evidence


def analyze_prospectus_row_single_question(row, question):
    # System and user prompts
    system_prompt = "You are an expert in analyzing bond prospectuses and identifying specific risk factors."

    # Format the user prompt using the row's data
    prompt = f"""
{system_prompt}

For the following question and text, judge whether the text is "Highly Relevant", "Somewhat Relevant", or "Not Relevant".

Question:
{question}

Text:
Subsubsection Title: {row['Subsubsection Title']}
Subsubsection Text: {row['Subsubsection Text']}


Please provide your answer in the following JSON format:

{{
  "Relevance": "Highly Relevant", "Somewhat Relevant", or "Not Relevant",
  "Evidence": "The exact phrases or sentences from the document that support your assessment; otherwise, leave blank."
}}

Note: Only provide the JSON response without any additional text.
"""
    # Run the prompt through the model
    response = llm.invoke(input=prompt)

    # Parse the response
    try:
        # Extract the Relevance and Evidence fields
        relevance, evidence_list = extract_fields(response)
        # Join multiple evidence items into a single string
        evidence = '; '.join(evidence_list)
    except Exception as e:
        relevance = "Parsing Error"
        evidence = ""

    # Combine relevance and evidence
    if relevance in ["Highly Relevant", "Somewhat Relevant"] and evidence:
        combined_answer = f"{relevance}: {evidence}"
    elif relevance in ["Highly Relevant", "Somewhat Relevant"]:
        combined_answer = relevance
    elif relevance == "Not Relevant":
        combined_answer = "Not Relevant"
    else:
        combined_answer = "Parsing Error"

    # For debugging
    if combined_answer == "Parsing Error":
        print("Parsing Error encountered. Response was:")
        print(response)

    return combined_answer


In [41]:
# Loop over each row in the DataFrame
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    for column_name, question in questions.items():
        # Check if the answer column is already filled
        if pd.notnull(df.at[index, column_name]) and df.at[index, column_name] != "":
            # Skip processing this row for this question
            continue
        combined_answer = analyze_prospectus_row_single_question(row, question)
        df.at[index, column_name] = combined_answer
        
    # Save progress every 50 rows
    if index % 35 == 0 and index != 0:
        df.to_csv('prospectuses_data_processed.csv', index=False)
        # Remove the break statement to process all rows
        break


# Save the final DataFrame
df.to_csv('prospectuses_data_processed.csv', index=False)

 43%|████▎     | 32/74 [02:27<03:13,  4.60s/it]
